[Страница нейронки](https://github.com/haoheliu/versatile_audio_super_resolution)

In [ ]:
# @title **Установить AudioSR** { display-mode: "form" }
from IPython.display import clear_output, Audio, display
from ipywidgets import Button
import os
import fnmatch

!git clone https://github.com/Bebra777228/ASR.git

# Optional
!conda create -n audiosr python=3.9; conda activate audiosr
# Install AudioLDM
!pip3 install audiosr==0.0.7

os.makedirs('input_audio', exist_ok=True)

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
# @title Обработка длинного аудио (>30s) + стерео { display-mode: "form" }

#@markdown ---

#@markdown **Загрузите ваше аудио в папку input**

#@markdown *Обратите внимание, файл должен иметь формат .mp3 и не должен содержать кириллицы, пробелов и спецсимволов*

#@markdown ---

#@markdown **Выберите модель:**
model = "basic" # @param ["basic", "speech"] {allow-input: true}

#@markdown **Количество шагов**
ddim = 50 # @param {type:"slider", min:10, max:250, step:5}

#@markdown ---

#@markdown **Размер сегмента**
segment = 10 # @param {type:"slider", min:5, max:30, step:1}

#@markdown ---

#clear_output()

def find_latest_wav_file(path, pattern='*.wav'):
    matches = []

    for root, dirnames, filenames in os.walk(path):
        for filename in fnmatch.filter(filenames, pattern):
            matches.append(os.path.join(root, filename))

    if matches:
        latest_file = max(matches, key=os.path.getctime)
        return latest_file
    else:
        return None

# Создаем папку для обработанных аудиофайлов
os.makedirs('preprocess_audio', exist_ok=True)
os.makedirs('inlist_files', exist_ok=True)
os.makedirs('output_process', exist_ok=True)
os.makedirs('outlist_files', exist_ok=True)
os.makedirs('output_audio', exist_ok=True)

# Получаем список всех mp3 файлов в папке input_audio
sourcefiles = [f for f in os.listdir('input_audio') if f.endswith('.mp3')]

for n, sourcefile in enumerate (sourcefiles, start=1):
    procfile = '{:05}'.format (n)

    # Разрезаем mp3 файл на куски по 10 секунд
    os.system (f'ffmpeg -hide_banner -i input_audio/{sourcefile} -filter_complex "[0:a]channelsplit=channel_layout=stereo[left][right]" \
    -map_metadata -1 -fflags +bitexact -ar 48000 -ac 1 -f segment -segment_time {segment} -map "[left]" preprocess_audio/{procfile}_%03d_left.wav \
    -map_metadata -1 -fflags +bitexact -ar 48000 -ac 1 -f segment -segment_time {segment} -map "[right]" preprocess_audio/{procfile}_%03d_right.wav \
    2>> ffmpeg_logs.txt')

    segments = sorted ([f for f in os.listdir('preprocess_audio') if f.endswith('.wav')])

    # Перечисляем файлы в список для AudioSR
    with open (f'inlist_files/{procfile}_list.txt', 'w+') as input_segments_list:
        for segment_file in segments:
            input_segments_list.write (f"preprocess_audio/{segment_file}\n")

    # Основная обработка
    !audiosr -il inlist_files/{procfile}_list.txt -s output_process/ --model_name {model} --ddim_steps {ddim} -gs 1 --suffix ""

    # Перечисляем файлы в список для FFmpeg
    with open (f'outlist_files/{procfile}_list_left.txt', 'w+') as leftlist, open (f'outlist_files/{procfile}_list_right.txt', 'w+') as rightlist:
        for f in [find_latest_wav_file ('output_process', pattern = segment) for segment in segments]:
            if f.endswith('_left.wav'):
                leftlist.write (f"file '../{f}'\n")
            if f.endswith('_right.wav'):
                rightlist.write (f"file '../{f}'\n")

    # Соединяем файлы обратно
    os.system(f'ffmpeg -hide_banner -f concat -safe 0 -i "outlist_files/{procfile}_list_left.txt" -f concat -safe 0 -i "outlist_files/{procfile}_list_right.txt" \
    -filter_complex "[0:a][1:a]amerge=inputs=2[a]" -map_metadata -1 -fflags +bitexact -ar 48000 -map "[a]" \
    output_audio/{sourcefile.replace(".mp3", ".flac")} 2>> ffmpeg_logs.txt')


In [39]:
# @title Очистка { display-mode: "form" }
#@markdown **Выполнить в случае ошибки**

!rm -rf preprocess_audio/*
!rm -rf inlist_files/*
!rm -rf output_process/*
!rm -rf outlist_files/*
!rm -rf output_audio/*
!rm ffmpeg_logs.txt